# 1. City (Web scraping)

1. Begin by scraping the country, the latitude and the longitude of each city from their respective Wikipedia pages:

Berlin: https://en.wikipedia.org/wiki/Berlin

Hamburg: https://en.wikipedia.org/wiki/Hamburg

Munich: https://en.wikipedia.org/wiki/Munich

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [5]:
def city_info(city_list):
    city_info = []
    for city in city_list:
        url = f'https://en.wikipedia.org/wiki/{city}' ## get the Wikipedia page for the city
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        country = soup.find(string='Country').find_next().get_text()
        latitude = soup.find(string='Coordinates: ').find_next().get_text().split()[-2].strip(';')
        longitute = soup.find(string='Coordinates: ').find_next().get_text().split()[-1].strip(';')
        population = int(soup.find(string = 'Population').find_next('td').get_text().replace(',', ''))
        city_info.append([city, country, latitude, longitute, population])
    return pd.DataFrame(city_info, columns = ['city', 'country', 'latitude', 'longitude', 'population'])

## Our sample cities
cities = ('Berlin', 'Hamburg', 'Munich') ## can be adjusted to any city you wish
cities_df = city_info(cities)
cities_df

,city,country,latitude,longitude,population
0,Berlin,Germany,52.52000,13.40500,3878100
1,Hamburg,Germany,53.550,10.000,1964021
2,Munich,Germany,48.13750,11.57500,1510378


In [6]:
schema = "gans_sample"
host = "localhost"
user = "root"
password = "password" ## MySQL password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [7]:
# cities_df.to_sql('cities',
#                   if_exists='append',
#                   con=connection_string,
#                   index=False)

3

In [8]:
cities_df = pd.read_sql('cities', con=connection_string)
cities_df

,city_id,city,country,latitude,longitude,population
0,1,Berlin,Germany,52.5200,13.405,3878100
1,2,Hamburg,Germany,53.5500,10.000,1964021
2,3,Munich,Germany,48.1375,11.575,1512491
3,4,Berlin,Germany,52.5200,13.405,3878100
4,5,Hamburg,Germany,53.5500,10.000,1964021
5,6,Munich,Germany,48.1375,11.575,1510378
6,7,Berlin,Germany,52.5200,13.405,3878100
7,8,Hamburg,Germany,53.5500,10.000,1964021
8,9,Munich,Germany,48.1375,11.575,1510378


In [129]:
cities_df = cities_df.iloc[0:3]

# 2. Weather

## 2.1. One city

In [131]:
berlin = cities_df.loc[0]
berlin

city_id             1
city           Berlin
country       Germany
latitude        52.52
longitude      13.405
population    3878100
Name: 0, dtype: object

In [133]:
API_key = 'api_key' ## Openweather API key

In [135]:
response_berlin = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={berlin['city']}&appid={API_key}&units=metric")
berlin_json = response_berlin.json()

In [137]:
berlin_json['list'][0]

{'dt': 1723906800,
 'main': {'temp': 27.95,
  'feels_like': 29.27,
  'temp_min': 25.63,
  'temp_max': 27.95,
  'pressure': 1010,
  'sea_level': 1010,
  'grnd_level': 1005,
  'humidity': 59,
  'temp_kf': 2.32},
 'weather': [{'id': 500,
   'main': 'Rain',
   'description': 'light rain',
   'icon': '10d'}],
 'clouds': {'all': 0},
 'wind': {'speed': 3.95, 'deg': 336, 'gust': 4.36},
 'visibility': 10000,
 'pop': 0.98,
 'rain': {'3h': 1.24},
 'sys': {'pod': 'd'},
 'dt_txt': '2024-08-17 15:00:00'}

In [115]:
berlin_json['list'][0]['main']['temp']
berlin_json['list'][0]['main']['temp_min']
berlin_json['list'][0]['main']['temp_max']
berlin_json['list'][0]['weather'][0]['main']
berlin_json['list'][0]['weather'][0]['description']
berlin_json['list'][0]['wind']['gust']
berlin_json['list'][0].get('rain', {}).get('3h')
berlin_json['list'][0]['dt_txt']

'2024-08-17 15:00:00'

In [117]:
temp = []
temp_min = []
temp_max = []
main = []
description = []
gust = []
rain = []
time = []
city_id = []

for forecast in berlin_json['list']:
    temp.append(forecast['main']['temp'])
    temp_min.append(forecast['main']['temp_min'])
    temp_max.append(forecast['main']['temp_max'])
    main.append(forecast['weather'][0]['main'])
    description.append(forecast['weather'][0]['description'])
    gust.append(forecast['wind']['gust'])
    rain.append(forecast.get('rain', {}).get('3h'))
    time.append(forecast['dt_txt'])
    city_id.append(berlin['city_id'])

In [119]:
pd.DataFrame({'temp': temp,
              'temp_min': temp_min,
              'temp_max': temp_max,
              'main': main,
              'description': description,
              'gust': gust,
              'rain': rain,
              'time': time,
              'city_id': city_id})

,temp,temp_min,temp_max,main,description,gust,rain,time,city_id
0,27.93,25.63,27.93,Rain,light rain,4.36,1.24,2024-08-17 15:00:00,1
1,26.58,23.88,26.58,Clouds,overcast clouds,4.20,NaN,2024-08-17 18:00:00,1
2,22.99,20.52,22.99,Rain,light rain,6.20,1.87,2024-08-17 21:00:00,1
3,18.87,18.87,18.87,Rain,light rain,5.45,1.29,2024-08-18 00:00:00,1
4,17.95,17.95,17.95,Rain,light rain,4.90,0.86,2024-08-18 03:00:00,1
5,18.87,18.87,18.87,Rain,light rain,4.51,0.56,2024-08-18 06:00:00,1
6,21.76,21.76,21.76,Clouds,overcast clouds,2.30,NaN,2024-08-18 09:00:00,1
7,23.74,23.74,23.74,Clouds,overcast clouds,2.18,NaN,2024-08-18 12:00:00,1
8,18.47,18.47,18.47,Rain,moderate rain,7.76,5.78,2024-08-18 15:00:00,1
9,16.44,16.44,16.44,Rain,moderate rain,7.71,3.60,2024-08-18 18:00:00,1


## 2.2. All city

In [139]:
cities_df

,city_id,city,country,latitude,longitude,population
0,1,Berlin,Germany,52.5200,13.405,3878100
1,2,Hamburg,Germany,53.5500,10.000,1964021
2,3,Munich,Germany,48.1375,11.575,1512491


In [143]:
## Select the info from the weather data that are relevant to us.
temp = []
temp_min = []
temp_max = []
main = []
description = []
gust = []
rain = []
time = []
city_id = []

for i, row in cities_df.iterrows():
    response = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={row['city']}&appid={API_key}&units=metric")
    if response.status_code == 200:
        response_json = response.json()
        for forecast in response_json['list']:
            temp.append(forecast['main']['temp'])
            temp_min.append(forecast['main']['temp_min'])
            temp_max.append(forecast['main']['temp_max'])
            main.append(forecast['weather'][0]['main'])
            description.append(forecast['weather'][0]['description'])
            gust.append(forecast['wind']['gust'])
            rain.append(forecast.get('rain', {}).get('3h'))
            time.append(forecast['dt_txt'])
            city_id.append(row['city_id'])

## Convert to the dataframe
weather_df = pd.DataFrame({'temp': temp,
                           'temp_min': temp_min,
                           'temp_max': temp_max,
                           'main': main,
                           'description': description,
                           'gust': gust,
                           'rain': rain,
                           'time': time,
                           'city_id': city_id})

weather_df

,temp,temp_min,temp_max,main,description,gust,rain,time,city_id
0,27.95,25.63,27.95,Rain,light rain,4.36,1.24,2024-08-17 15:00:00,1
1,26.59,23.88,26.59,Clouds,scattered clouds,4.20,NaN,2024-08-17 18:00:00,1
2,23.00,20.52,23.00,Rain,light rain,6.20,1.87,2024-08-17 21:00:00,1
3,18.87,18.87,18.87,Rain,light rain,5.45,1.29,2024-08-18 00:00:00,1
4,17.95,17.95,17.95,Rain,light rain,4.90,0.86,2024-08-18 03:00:00,1
5,18.87,18.87,18.87,Rain,light rain,4.51,0.56,2024-08-18 06:00:00,1
6,21.76,21.76,21.76,Clouds,overcast clouds,2.30,NaN,2024-08-18 09:00:00,1
7,23.74,23.74,23.74,Clouds,overcast clouds,2.18,NaN,2024-08-18 12:00:00,1
8,18.47,18.47,18.47,Rain,moderate rain,7.76,5.78,2024-08-18 15:00:00,1
9,16.44,16.44,16.44,Rain,moderate rain,7.71,3.60,2024-08-18 18:00:00,1


## 2.3. Sending to SQL

In [22]:
# weather_df.to_sql('weather',
#                   if_exists='append',
#                   con=connection_string,
#                   index=False)

360

In [23]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   temp         360 non-null    float64
 1   temp_min     360 non-null    float64
 2   temp_max     360 non-null    float64
 3   main         360 non-null    object 
 4   description  360 non-null    object 
 5   gust         360 non-null    float64
 6   rain         111 non-null    float64
 7   time         360 non-null    object 
 8   city_id      360 non-null    int64  
dtypes: float64(5), int64(1), object(3)
memory usage: 25.4+ KB


# 3. Airport

In [28]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

querystring = {"lat":"52.31","lon":"13.24","radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

headers = {
	"x-rapidapi-key": "API_key", ## API key
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'searchBy': {'lat': 52.31, 'lon': 13.24}, 'count': 1, 'items': [{'icao': 'EDDB', 'iata': 'BER', 'name': 'Berlin Brandenburg', 'shortName': 'Brandenburg', 'municipalityName': 'Berlin', 'location': {'lat': 52.35139, 'lon': 13.493889}, 'countryCode': 'DE', 'timeZone': 'Europe/Berlin'}]}


In [26]:
response.json()

{'message': 'You are not subscribed to this API.'}

In [30]:
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889


In [31]:
def icao_airport_codes(df):
    list_for_df = []
    for index, row in df.iterrows():
        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        
        ## we will get airports info by using 'latitude', 'longitude' info of each city.
        querystring = {"lat":str(row['latitude']),"lon":str(row['longitude']),"radiusKm":"30","limit":"5","withFlightInfoOnly":"true"}
        
        headers = {
         "x-rapidapi-key": "API_Key", ## your API key here
         "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }
        
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            response_json = response.json()
            response_json['items'][0].update({'city_id': row['city_id']})
            list_for_df.append(pd.json_normalize(response_json['items']))

    return pd.concat(list_for_df, ignore_index=True)

airports_df = icao_airport_codes(cities_df)
airports_df

In [32]:
airports_df = icao_airport_codes(cities_df)

In [48]:
# airports_df[['icao', 'iata', 'name', 'city_id']].to_sql(
#     'airports',
#     if_exists='append',
#     con=connection_string,
#     index=False)

In [49]:
airports_df = pd.read_sql('airports', con=connection_string)
airports_df

,icao,iata,name,city_id
0,EDDB,BER,Berlin Brandenburg,1
1,EDDH,HAM,Hamburg,2
2,EDDM,MUC,Munich,3


# 4. Flight

## 4.1. Time set up for tomorrow

In [52]:
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{EDDB}/{2024-08-13}T{12}:00/{2024-08-13}T{23}:59"

querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
	"x-rapidapi-key": "API",
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (1922455839.py, line 1)

In [67]:
(pd.Timestamp.now()+pd.Timedelta(24, 'hours')).strftime("%Y-%m-%d")

'2024-08-18'

In [68]:
times = [['00', '11'], ['12', '23']]

In [146]:
API_key = "API"
tomorrow = (pd.Timestamp.now() + pd.Timedelta(24, 'hours')).strftime('%Y-%m-%d')
times = [['00', '11'], ['12', '23']]
for index, row in airports_df.iloc[0:1].iterrows():
    for time in times[0:1]:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['icao']}/{tomorrow}T{time[0]}:00/{tomorrow}T{time[1]}:59"
        
        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
        
        headers = {
        	"x-rapidapi-key": '69f53b2cd8mshbecc722006fd546p162db0jsne262715df6cd',
        	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }
        
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            response_json = response.json()
        else:
            print(f'Error at {row["icao"]}: {response.message}')

## 4.2. Arrival info

In [71]:
response_json.keys()

dict_keys(['arrivals'])

In [72]:
len(response_json['arrivals'])

88

In [73]:
response_json['arrivals'][0]

{'movement': {'airport': {'icao': 'LTBJ',
   'iata': 'ADB',
   'name': 'İzmir',
   'timeZone': 'Europe/Istanbul'},
  'scheduledTime': {'utc': '2024-08-18 04:30Z',
   'local': '2024-08-18 06:30+02:00'},
  'revisedTime': {'utc': '2024-08-18 04:30Z',
   'local': '2024-08-18 06:30+02:00'},
  'terminal': '1',
  'gate': 'Y03',
  'baggageBelt': 'B3',
  'quality': ['Basic', 'Live']},
 'number': 'XQ 966',
 'status': 'Expected',
 'codeshareStatus': 'IsOperator',
 'isCargo': False,
 'aircraft': {'model': 'Boeing 737-700 (winglets)'},
 'airline': {'name': 'Sun Express', 'iata': 'XQ', 'icao': 'SXS'}}

In [74]:
response_json['arrivals'][0].keys()

dict_keys(['movement', 'number', 'status', 'codeshareStatus', 'isCargo', 'aircraft', 'airline'])

In [75]:
response_json['arrivals'][0]['movement']['airport']['name']
response_json['arrivals'][0]['movement']['airport']['icao']
response_json['arrivals'][0]['movement'].get('revisedTime', response_json['arrivals'][0]['movement']['scheduledTime'])['local']
response_json['arrivals'][0]['number']
response_json['arrivals'][0]['aircraft']['model']
response_json['arrivals'][0]['airline']['name']

'Sun Express'

In [76]:
## Select the info from the data that are relevant to us.
flight_dict = {'arrival_icao': [],
               'departing': [],
               'departure_icao': [],
               'arrival_time': [],
               'flight_number': [],
               'model': [],
               'airline': []
              }

for index, row in airports_df.iterrows():
    for time in times:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['icao']}/{tomorrow}T{time[0]}:00/{tomorrow}T{time[1]}:59"
        
        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
        
        headers = {
         "x-rapidapi-key": 'API_KEY', ## You need your own paid API here
         "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }
        
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            response_json = response.json()
            for flight in response_json['arrivals']:
                flight_dict['arrival_icao'].append(row['icao'])
                flight_dict['departing'].append(flight['movement']['airport']['name'])
                flight_dict['departure_icao'].append(flight['movement']['airport'].get('icao'))
                flight_dict['arrival_time'].append(flight['movement'].get('revisedTime', flight['movement']['scheduledTime'])['local'])
                flight_dict['flight_number'].append(flight['number'])
                flight_dict['model'].append(flight.get('aircraft', {}).get('model'))
                flight_dict['airline'].append(flight.get('airline', {}).get('name')) 
        else:
            print(f'Error at {row["icao"]}: {response.message}')
flights_df = pd.DataFrame(flight_dict)
flights_df['arrival_time'] = pd.to_datetime(flights_df['arrival_time'])
flights_df

## 4.3. Merge both info

In [150]:
API_key = "API"
tomorrow = (pd.Timestamp.now() + pd.Timedelta(24, 'hours')).strftime('%Y-%m-%d')
times = [['00', '11'], ['12', '23']]
flight_dict = {'arrival_icao': [],
               'departing': [],
               'departure_icao': [],
               'arrival_time': [],
               'flight_number': [],
               'model': [],
               'airline': []
              }
for index, row in airports_df.iterrows():
    for time in times:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['icao']}/{tomorrow}T{time[0]}:00/{tomorrow}T{time[1]}:59"
        
        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
        
        headers = {
        	"x-rapidapi-key": '69f53b2cd8mshbecc722006fd546p162db0jsne262715df6cd',
        	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }
        
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            response_json = response.json()
            for flight in response_json['arrivals']:
                flight_dict['arrival_icao'].append(row['icao'])
                flight_dict['departing'].append(flight['movement']['airport']['name'])
                flight_dict['departure_icao'].append(flight['movement']['airport'].get('icao'))
                flight_dict['arrival_time'].append(flight['movement'].get('revisedTime', flight['movement']['scheduledTime'])['local'])
                flight_dict['flight_number'].append(flight['number'])
                flight_dict['model'].append(flight.get('aircraft', {}).get('model'))
                flight_dict['airline'].append(flight.get('airline', {}).get('name')) 
        else:
            print(f'Error at {row["icao"]}: {response.message}')
flights_df = pd.DataFrame(flight_dict)
flights_df['arrival_time'] = pd.to_datetime(flights_df['arrival_time'])
flights_df.head(10)

,arrival_icao,departing,departure_icao,arrival_time,flight_number,model,airline
0,EDDB,İzmir,LTBJ,2024-08-18 06:30:00+02:00,XQ 966,Boeing 737-700 (winglets),Sun Express
1,EDDB,Kayseri,LTAU,2024-08-18 06:45:00+02:00,XQ 1716,Boeing 737-700 (winglets),Sun Express
2,EDDB,Beijing,ZBAA,2024-08-18 06:45:00+02:00,HU 489,Airbus A330-300,Hainan
3,EDDB,Newark,KEWR,2024-08-18 07:15:00+02:00,UA 962,Boeing 767-400(ER),United
4,EDDB,Doha,OTHH,2024-08-18 07:20:00+02:00,QR 79,Boeing 787-9,Qatar Airways
5,EDDB,Chisinau,None,2024-08-18 07:20:00+02:00,OE 611,Airbus A320,OE
6,EDDB,Chisinau,None,2024-08-18 07:20:00+02:00,5F 611,Airbus A320,FlyOne
7,EDDB,Stuttgart,EDDS,2024-08-18 07:25:00+02:00,EW 2000,Airbus A319,Eurowings
8,EDDB,Trieste,LIPQ,2024-08-18 07:35:00+02:00,FR 6781,Boeing 737 MAX 8,Ryanair
9,EDDB,Yerevan,UDYZ,2024-08-18 07:45:00+02:00,EW 8995,Airbus A320-200 (sharklets),Eurowings


In [79]:
# flights_df.to_sql(
#     'flights',
#     if_exists='append',
#     con=connection_string,
#     index=False)